In [ ]:
#importing libraries
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
input_dir = "image-processing-files/test_images/"
output_dir = "image-processing-files/results/"


In [ ]:
#3 images for comparison.
#image 5 because got a large bit of brightness noise, but other than that reasonably dark
im05= cv2.imread('image-processing-files/test_images/im05-RET039OD.jpg', cv2.IMREAD_COLOR)
#image 27 because bright and salt and pepper noise
im27= cv2.imread('image-processing-files/test_images/im27-RET051OD.jpg', cv2.IMREAD_COLOR)
#image 12 because lack of contrast and different noise 
im12= cv2.imread('image-processing-files/test_images/im12-RET137OD.jpg', cv2.IMREAD_COLOR)
#image 3 goes black screen after warping and inpainting:
im03= cv2.imread('image-processing-files/test_images/im03-RET033OD.jpg', cv2.IMREAD_COLOR)


In [ ]:
for filename in os.listdir(input_dir):
    img = cv2.imread(os.path.join(input_dir, filename))
    hist = cv2.calcHist([img], [0], None, [256], [0,256])
    plt.xlabel('Pixel value')
    plt.ylabel('Count')
    plt.title('Histogram of pixel values for all images')
    plt.plot(hist)
plt.savefig('Pixel values histogram')

In [ ]:
colour = ('b','g','r')
for filename in os.listdir(input_dir):
    img = cv2.imread(os.path.join(input_dir, filename))
    if img is not None:  # Add this check
        for i,col in enumerate(colour):
            histr = cv2.calcHist([img],[i],None,[255],[1,255])
            plt.plot(histr,color = col)
            plt.xlim([1,256])
            plt.xlabel('Pixel value')
            plt.ylabel('Count')
        
plt.show()


In [ ]:
#inpainting finally works!!!!!
processed_imgs = []
for img in [im05, im12, im27, im03]:
    mask = np.zeros_like(img[:,:,0])
    center = (188, 212)
    radius = 22
    cv2.circle(mask, center, radius, 255, -1)

    # Extract a similar section from the image
    section = img[120:160, 190:230]

    # Resize the mask to match the size of the image using interpolation
    mask_resized = cv2.resize(mask, img.shape[:2][::-1], interpolation=cv2.INTER_LANCZOS4)

    # Inpaint the corrupted region in the color image using the extracted section
    inpainted_color = cv2.inpaint(img, mask_resized, 3, cv2.INPAINT_TELEA)

    # Resize the section to match the size of the inpainted region
    section_resized = cv2.resize(section, inpainted_color.shape[:2][::-1], interpolation=cv2.INTER_LANCZOS4)

    # Replace the inpainted region with the extracted section
    processed_img = inpainted_color.copy()
    processed_img[mask_resized != 0] = section_resized[mask_resized != 0]

    # Append the processed image to the list
    processed_imgs.append(processed_img)


In [ ]:
#dewarping and removing borders 1
dewarped = []
for img in processed_imgs:
    img=img
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 10, 255, 0)

    plt.imshow(thresh)
    plt.show()
    #dewarping and removing borders 2
    # finding the contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE,
                                cv2.CHAIN_APPROX_SIMPLE)
    
    # take the first contour
    cnt = contours[0]

    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    box_img = cv2.drawContours(img, [box], 0, (0, 0, 255), 2)

    # Show the result
    plt.imshow(box_img)
    plt.show()
    #dewarping and removing borders 3

    extent = np.float32([[0,0], [255,0], [255,255], [0,255]])
    bounds = np.float32(box)

    M = cv2.getPerspectiveTransform(bounds, extent)

    # Apply the transformation to the image
    result = cv2.warpPerspective(img, M, (256, 256))
    result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    # Show the result
    plt.imshow(result)
    plt.show()
    dewarped.append(result)

In [ ]:
for img in dewarped:
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Create a structuring element for morphological operations
    kernel = np.ones((5,5), np.uint8)

    # Apply dilation to the image
    dilation = cv2.dilate(img, kernel, iterations=1)

    # Apply opening to the image
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

    # Apply closing to the image
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    #Apply median to the image 
    median = cv2.medianBlur(img, 5)

    # Apply bilateral to the image
    bilateral = cv2.bilateralFilter((img),3,75,75)

    # Display the original and processed images
    cv2.imshow('Original Image', img)
    cv2.imshow('Dilation', dilation)
    cv2.imshow('Opening', opening)
    cv2.imshow('Closing', closing)
    cv2.imshow('Median', median)
    cv2.imshow('Bilateral', bilateral)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# median
median_list = []
for img in dewarped:
    median = cv2.medianBlur(img, 5)
    plt.imshow(median)
    plt.show()
    median_list.append(median)



In [ ]:
color_list = []
for img in contrast_list:
    # Convert the image to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Split the LAB image into separate channels
    l_channel, a_channel, b_channel = cv2.split(lab)

    # Normalize the L channel to increase contrast
    l_channel = cv2.normalize(l_channel, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    # Adjust the brightness of the A and B channels
    a_channel = cv2.addWeighted(a_channel, 0.9, np.mean(a_channel), 0, 0)
    b_channel = cv2.addWeighted(b_channel, 0.9, np.mean(b_channel), 0, 0)

    # Merge the channels back into a single LAB image
    lab = cv2.merge([l_channel, a_channel, b_channel])

    # Convert the LAB image back to BGR color space
    balanced = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # Display the original and balanced images side-by-side
    plt.imshow(img)
    plt.imshow(balanced)
    plt.show()
    color_list.append(balanced)
   

In [ ]:
#MEAN PIXEL VALUE FOR BRIGHTNESS

#almost no need for colour changing- just ruining my results
# Define the directory containing the images

# Define the target mean pixel value
target_mean = 120
contrast_list=[]
# Loop over all images in the directory
for img in median_list:
    # Load the image and convert to grayscale
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Compute the mean pixel value
    mean_pixel = np.mean(gray)

    # Compute the scale factor
    scale_factor = target_mean / mean_pixel
    difference = abs(target_mean-mean_pixel)
    # Adjust the contrast and brightness
    adjusted = cv2.convertScaleAbs(img, alpha=scale_factor, beta=difference)

    # Save the adjusted image
    plt.imshow(img)
    plt.imshow(adjusted)
    plt.show()
    contrast_list.append(adjusted)


In [ ]:
for img in contrast_list:
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    # Increase the contrast and brightness
    alpha = 0.4 # Contrast control (1.0-3.0)
    beta = 60  # Brightness control (0-100)
    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    # Display the original and adjusted images side-by-side
    cv2.imshow('Original', img)
    cv2.imshow('Adjusted', adjusted)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
for img in median_list:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection
    edges = cv2.Canny(gray, 10, 50)

    # Apply Gaussian blur to remove noise
    blur = cv2.GaussianBlur(img, (5, 5), 0)

    # Create a sharpening filter
    kernel = np.array([[-1,-1,-1],
                    [-1, 9,-1],
                    [-1,-1,-1]])

    # Apply the unsharp masking filter to the edges only
    sharp_edges = cv2.bitwise_and(blur, blur, mask=edges)
    sharp_edges = cv2.filter2D(sharp_edges, -1, kernel)

    # Combine the sharpened edges with the original image
    result = cv2.addWeighted(img, 2, sharp_edges, -0.5, 0)
    result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    # Display the result
    cv2.imshow('Original Image', img)
    cv2.imshow('Sharpened Image with Edges', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [23]:
for img in median_list:
    gamma = 3.0
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype('uint8')
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    # Apply the exponential transform to the image
    img_enhanced = cv2.LUT(img, table)
    
    # Save the enhanced image to disk

    cv2.imshow('Edges',img_enhanced )
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [37]:
for img in median_list:
    # Convert the image to the LAB color space
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Split the LAB image into separate channels
    l_channel, a_channel, b_channel = cv2.split(img_lab)

    # Apply adaptive histogram equalization to the L channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l_channel_eq = clahe.apply(l_channel)

    # Merge the equalized L channel with the original A and B channels
    img_lab_eq = cv2.merge((l_channel_eq, a_channel, b_channel))

    # Convert the image back to the original color space
    img_output = cv2.cvtColor(img_lab_eq, cv2.COLOR_LAB2RGB)
    cv2.imshow('Edges',img_output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [52]:
for img in median_list:
    # gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Perform unsharp masking
    blurred_img = cv2.GaussianBlur(img, (0, 0), 5)
    mask = cv2.subtract(img, blurred_img)
    unsharp_img = cv2.addWeighted(img, 1.5, mask, 0, 0)
    # Convert back to color
    color_img = cv2.cvtColor(unsharp_img, cv2.COLOR_BGR2RGB)
    cv2.imshow('Edges',color_img )
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
for img in median_list:
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    adjusted= cv2.filter2D(img, -1, kernel)

    # Display the original and adjusted images side-by-side
    cv2.imshow('Original', img)
    cv2.imshow('Adjusted', adjusted)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [40]:
for img in median_list:
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

    # Apply ACE
    ace_img = clahe.apply(img)
    ace_img= cv2.cvtColor(ace_img, cv2.COLOR_GRAY2RGB)
    cv2.imshow('Edges',ace_img )
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
#nmeans for image 1 
nmeans_1 = cv2.fastNlMeansDenoisingColored(im01)
cv2.imwrite("nmeans_1.jpg", nmeans_1)

#Bilateral for image 1 
bilat_1 = cv2.bilateralFilter((im01),3,75,75)
cv2.imwrite("bilat_1.jpg", bilat_1)

#Gaussian on image 1 
gaussian_1 = cv2.GaussianBlur(im01, (3,3), 30)
cv2.imwrite("gaussian_1.jpg", gaussian_1)

#exp function
import math
def exponential_transform(image, c, alpha):
    for i in range(0, image.shape[1]):  # image width
        for j in range(0, image.shape[0]):  # image height

            # compute exponential transform

            image[j, i] = int(c * (math.pow(1 + alpha, image[j, i]) - 1))
    return image
scaled_1= cv2.resize(im01, ((int(im01.shape[1] * 0.6)),int(im01.shape[0] * 0.6)), interpolation=cv2.INTER_AREA)
gray_scaled_im01 = cv2.cvtColor(scaled_1, cv2.COLOR_BGR2GRAY)
EXP1 = exponential_transform(gray_scaled_im01, 130, 0.003)
colored_EXP1 = cv2.cvtColor(EXP1, cv2.COLOR_GRAY2BGR)
cv2.imwrite("EXPt1.png", colored_EXP1)

# gamma correction 
import numpy as np
gamma_1 = np.array(255*(1/255)**1.4,dtype='uint8')
cv2.imwrite("gamma_1.png", gamma_1)